# Tiniest Blockchain Example Notebook
Original source found: https://medium.com/crypto-currently/lets-build-the-tiniest-blockchain-e70965a248b



In [5]:
#Import packages
import hashlib as hasher
import datetime as date

## SHA Hashing Example
SHA256 hash provides unique alpha numeric code (i.e. "hash") for given value. Able to recreate original value from given hash.

Example below

In [7]:
sha = hasher.sha256()
print(sha.hexdigest())

e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855



Above shows a random sha256 'hash' - an alpha numeric output of fixed length

Hash works in practice by taking a 'string' - an alpha numeric input of ANY length - and returning fixed length, deterministic hash

In [11]:
sha.update("1235asihgasdh".encode("utf-8"))
print(sha.hexdigest())

19a02fb23ed4f4d29228b1b2b318cc37f3efdb20859048d3397ff4c57f55dbe6



'Deterministic' means same input will always result in the same output

Below we take the same input as before and reproduce the hash value


In [14]:
print(hasher.sha256("1235asihgasdh".encode("utf-8")).hexdigest())

19a02fb23ed4f4d29228b1b2b318cc37f3efdb20859048d3397ff4c57f55dbe6



It's important to note, the hash function does not work in reverse. You cannot determine what data formed the hash function's input just because you know the hash function's output. 



In blockchain, each new block (Block 1) includes the hash of the previous block's (Block 0) data AND its hash. 

Because the previous block's (Block 0) hash includes it's previous block (block -1), once a new block is created, the deterministic hash function prevents all subsequent blocks from being altered. To modify the 'chain' of blocks and hashes, someone would have to modify every single block.

Even without encryption modifying every block is a monumental undertaking.

Combined with encryption and have multiple copies of your blockchain, all owned by different users, all regularly checking the integrity of their transaction chain, a 'distributed ledger' is possible.

However, the technology enabled by the blockchain concept can be difficult to understand without a strong understanding of how blockchain is created.




## Creating Blockchain
Blockchain is series of sha or other hashes of data. Hash for given data must also include previous hash.

Provides 'chain' of data history. All hashes contain entire previous history of data; therefore historically immutable when distributed (i.e. stored in multiple locations)

As a refresher:
* Each block has data, a timestamp, possible index, and the previous block's hash
* The previous block's hash and data aare is used when creating it's hash for the current block
* The sequence of hashes creates a full history of transactions
* hash0 -> txn1 -> hash(txn1, hash0)-> hash1 -> tnx2 -> hash(tnx2, hash1) -> hash2

## Building blocks

The code below creates a block 'class' - a template for all future blocks which holds the block's data AND any operations we want to use only within that class. The operations are called 'methods' and give us ways to perform repeatable operations that users of the class cannot manipulate.

But that detail isn't important, and methods, classes and functions is confusing for those getting started,so let's not dwell on it. The import parts are:
* creating a block requires an index, timestamp, data, and the previous hash as inputs
* the method 'hash_block' returns the sha256 hash for this block

'hash_block' is eactly the same operation we saw above, only instead of a typed string, it's building the hash using the values assigned when creating the block


In [24]:
class Block:
    def __init__(self, index, timestamp, data, previous_hash):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.hash = self.hash_block()
        
    def hash_block(self):
        sha = hasher.sha256()
        hash_string = str(self.index) + str(self.timestamp) +  str(self.data) + str(self.previous_hash)
        sha.update(hash_string.encode("utf-8"))
        return sha.hexdigest()


Proof from prior example:

In [28]:
#using our new class
exp_block = Block("","","", "1235asihgasdh")
print(exp_block.hash_block())

#using the previous exammple:
print(hasher.sha256("1235asihgasdh".encode("utf-8")).hexdigest())

19a02fb23ed4f4d29228b1b2b318cc37f3efdb20859048d3397ff4c57f55dbe6
19a02fb23ed4f4d29228b1b2b318cc37f3efdb20859048d3397ff4c57f55dbe6


### First Block
First transaction constructs first block hash. Function instantiates an object of Block class defined in previous module. Block class object has zero index, arbitrary data of "Gensis Block" and arbitrary hash of  "0". 

In [29]:
def create_genesis_block():
    return Block(0, date.datetime.now(), "Genesis Block", "0")
    

### Next Block
Define rules around instantiated subsequent Block objects. Requires A previous block to instantiate a new block

In [30]:
def next_block(last_block):
    this_index = last_block.index + 1
    this_timestamp = date.datetime.now()
    this_data = "Hey! I'm block " + str(this_index)
    this_previous_hash = last_block.hash
    return Block(this_index, this_timestamp, this_data, this_previous_hash)

## Building the Blockchain

Construct a Blockchain List with sample data. List is indexed series of blocks using previous blocks sha hash

In [32]:
## Demonstration of list vs string, not part of application
bc_string = create_genesis_block()
bc_list = [create_genesis_block()]

print(bc_string.hash_block())

#has index positions
print(bc_list[0].hash_block())

bc_list.append(bc_string.hash_block())

print(bc_list[1])

4f7ad96c48b376be6b05252445e63a8106e291500f3301aad19b27fec5e60a96
4f7ad96c48b376be6b05252445e63a8106e291500f3301aad19b27fec5e60a96
4f7ad96c48b376be6b05252445e63a8106e291500f3301aad19b27fec5e60a96


In [34]:
## actual demostration

blockchain = [create_genesis_block()]
previous_block = blockchain[0]

# number of blocks to create in list

num_of_blocks_to_add = 20 

# adding blocks to python list i.e. "chain"

for i in range(0, num_of_blocks_to_add):
    block_to_add = next_block(previous_block)
    blockchain.append(block_to_add)
    previous_block = block_to_add
    
    print("Block #{} has been added to blockchain!".format(block_to_add.index))
    print("Hash: {}\n".format(block_to_add.hash))



Block #1 has been added to blockchain!
Hash: 276a2caf726dc113392739521e143d29f5bdb179c258592b5eb21c4e6fd24265

Block #2 has been added to blockchain!
Hash: 7f3f0f2ef4396da0c89c8ff455a371512dcc0dccd288104cfed88a8caa49f22e

Block #3 has been added to blockchain!
Hash: 89a1d6f8f46a5398ab6784f46f037d2a3398b3610d2dde52fabe4de5d15ed854

Block #4 has been added to blockchain!
Hash: 7fb338005bbfbd21b3d1cc15fef217a22ac571356a25ca1b671770f85db43497

Block #5 has been added to blockchain!
Hash: 049cef06a087ef3ec6dd370c91e9bc4ab89e9256488a16fcc23a1886dec7bf4e

Block #6 has been added to blockchain!
Hash: 255cd84f9a674d23fbae9b1fc4d5dd9c684fbc2030020c00721337d7f7fc5748

Block #7 has been added to blockchain!
Hash: 17bcedb5eaa6af64002cc6e382b8eae42ab3e01fdab4d9b098fafad2eabbdd13

Block #8 has been added to blockchain!
Hash: 768508a865c0cd70bae6606b62e53225f4330346afb46d9fd32f4e5f9f4cc163

Block #9 has been added to blockchain!
Hash: 649779d2a3a43b3c230c3b92b4cd6ce4fa83747ca5ed2bb56eec45c8b13fc949

B

In [35]:
print(blockchain[15].data)
print(blockchain[15].hash)
print(blockchain[16].previous_hash)

Hey! I'm block 15
37ad9a3a3d9af2754859e543c754cd76e3803d8e2986e69d274483707e63019f
37ad9a3a3d9af2754859e543c754cd76e3803d8e2986e69d274483707e63019f


## Real World Example

Retail store needs to track shipments through complex supply chain. e.g. Walmart fruit goes through many different third parties before arriving in store.

Blockchain gives complete picture of all transactions without requiring all 3rd parties to access central database

### Creating Functions

In [36]:
def next_block(last_block, data):
    this_index = last_block.index + 1
    this_timestamp = date.datetime.now()
    this_data = data
    this_previous_hash = last_block.hash
    return Block(this_index, this_timestamp, this_data, this_previous_hash)

In [37]:
def shipment(buyer, seller, invoice_id):
    shipment = {"from": seller,
               "to": buyer,
               "invoice_id": invoice_id}
    return shipment

In [38]:
def get_previous_block(current_blockchain):
    block = current_blockchain[len(current_blockchain)-1]
    return block

In [39]:
#example
# Python indexes starting with zero
# However, counts starting with one

print(len(blockchain))
print(blockchain[len(blockchain)-1].hash)

21
6d270246235d7d5a3bac437f0e65167c14d2f84a7481d536a5498c87519d6c74


In [40]:
def add_shipment_to_blockchain(shipment_info, current_blockchain):
    previous_block = get_previous_block(current_blockchain)
    new_block = next_block(previous_block, shipment_info)
    current_blockchain.append(new_block)
    return current_blockchain

In [41]:
# quality of life function to show data in blockchain

def print_data(current_blockchain):
    for i in current_blockchain:
        print(str(i.data) +" " + i.hash)

### Building the blockchain

In [42]:
blockchain = [create_genesis_block()]

In [43]:
# Printing data in blockchain
print_data(blockchain)

Genesis Block 47ce78d6b15146aed04c900ed68eb985088f3eb314974990ebfa6dac76b7919b


In [44]:
# Add transaction to chain
txn1 = shipment("Farm123", "Store234", "1")

In [45]:
blockchain = add_shipment_to_blockchain(txn1, blockchain)

In [46]:
print_data(blockchain)

Genesis Block 47ce78d6b15146aed04c900ed68eb985088f3eb314974990ebfa6dac76b7919b
{'from': 'Store234', 'to': 'Farm123', 'invoice_id': '1'} 090a7316ec556dcc4ad34fbf31ee50dac956c95a2ea798b627a9524002bd860e


In [47]:
txn2 = shipment("Store234", "Store567", "1")

In [48]:
blockchain = add_shipment_to_blockchain(txn1, blockchain)
print_data(blockchain)

Genesis Block 47ce78d6b15146aed04c900ed68eb985088f3eb314974990ebfa6dac76b7919b
{'from': 'Store234', 'to': 'Farm123', 'invoice_id': '1'} 090a7316ec556dcc4ad34fbf31ee50dac956c95a2ea798b627a9524002bd860e
{'from': 'Store234', 'to': 'Farm123', 'invoice_id': '1'} e76dbee12f838063eefee8a494c4a85b6b83a77e57a69f71ea6b66a3f42a5f96
